<a href="https://colab.research.google.com/github/akinoriosamura/tensorflow2.0-sample/blob/master/tf2_classify_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 50kB/s 


In [19]:
!pip uninstall -q -y tensorflow==2.0.0-beta1

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals


import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

GPU is NOT AVAILABLE


In [20]:
# !nvcc --version
print(tf.__version__)
tf.test.gpu_device_name()

2.0.0-beta1


''

In [0]:
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", split=(train_validation_split, tfds.Split.TEST), as_supervised=True)

In [0]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [46]:
train_example_batch

<tf.Tensor: id=430, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [31]:
train_labels_batch

<tf.Tensor: id=431, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [33]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:3])

<tf.Tensor: id=777, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [37]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
         
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
history = model.fit(
    train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20


W0619 05:43:03.479731 140311984920448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 6s 197ms/step - loss: 0.7852 - accuracy: 0.4584 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 5s 163ms/step - loss: 0.6473 - accuracy: 0.6284 - val_loss: 0.6108 - val_accuracy: 0.6709
Epoch 3/20
30/30 [==============================] - 5s 164ms/step - loss: 0.5870 - accuracy: 0.6991 - val_loss: 0.5663 - val_accuracy: 0.7192
Epoch 4/20
30/30 [==============================] - 5s 164ms/step - loss: 0.5439 - accuracy: 0.7389 - val_loss: 0.5311 - val_accuracy: 0.7500
Epoch 5/20
30/30 [==============================] - 5s 166ms/step - loss: 0.5058 - accuracy: 0.7701 - val_loss: 0.4981 - val_accuracy: 0.7720
Epoch 6/20
30/30 [==============================] - 5s 164ms/step - loss: 0.4684 - accuracy: 0.7988 - val_loss: 0.4664 - val_accuracy: 0.7941
Epoch 7/20
30/30 [==============================] - 5s 164ms/step - loss: 0.4318 - accuracy: 0.8239 - val_loss: 0.4367 - val_accuracy: 0.8109
Epoch 8/2

In [45]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.311
accuracy: 0.868
